In [ ]:
!pip install spotipy

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [ ]:
client_id = 'YOUR_SPOTIFY_CLIENT_ID'
client_secret = 'YOUR_SPOTIFY_CLIENT_SECRET'

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)


In [ ]:
genres = ['pop', 'rock', 'classical', 'hip hop', 'jazz']
features_list = []

for genre in genres:
    results = sp.search(q=f'genre:{genre}', type='track', limit=20)
    for track in results['tracks']['items']:
        audio_features = sp.audio_features(track['id'])[0]
        if audio_features:
            audio_features['genre'] = genre
            features_list.append(audio_features)

df = pd.DataFrame(features_list)
df.head()


In [ ]:
selected_features = [
    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'genre'
]

df = df[selected_features]


In [ ]:
le = LabelEncoder()
df['genre_encoded'] = le.fit_transform(df['genre'])

X = df.drop(['genre', 'genre_encoded'], axis=1)
y = df['genre_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))


In [ ]:
# Example: Billie Eilish - bad guy
track = sp.search(q="bad guy billie eilish", type="track", limit=1)['tracks']['items'][0]
features = sp.audio_features(track['id'])[0]

input_features = [[
    features['danceability'], features['energy'], features['key'], features['loudness'],
    features['mode'], features['speechiness'], features['acousticness'],
    features['instrumentalness'], features['liveness'], features['valence'], features['tempo']
]]

predicted_genre = le.inverse_transform(model.predict(input_features))[0]
print("Predicted Genre:", predicted_genre)
